In [1]:
import os
import re
import pandas as pd
import numpy as np
from natsort import natsorted, index_natsorted, order_by_index

part_pattern = "(^|ст(\.|))\s*\d{3}(\.\d{1}|)(\s*ч*\.*\s*\d{1}|)(\s*\.\d{1}|)"

def dropFirstRows(df, value2search, rowsNum = None):
    """
    Функция находит строку первого столбца в таблице, значение которой соответствует параметру value2search,
    и удаляет количество строк, указанное в параметре rowsNum.
    Если rowsNum не задан - удаляются все строки начиная с самой верхней.
    """
    for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
        if val == True and val != np.nan and index !=0:
            if rowsNum is None:
                df.drop(df.index[0:index], inplace=True)
            else:
                df.drop(df.index[index-rowsNum:index], inplace=True)
                break
            df.reset_index(drop=True, inplace=True)
    return df

def cutNumbers(string):
    '''Служебная функция нужная, чтобы при склеивании строк в tackleMergedCells вырезать строки, содержащие только числа'''
    string = str(string)
    string = string.replace("nan", "")
    match = re.search(r'(^\d+$)', string)
    if match:
        string = string.replace(match.group(), "")
    return string



def deleteUnusedCols(df, year):
    """
    Удаляет пустые колонки, а также неиспользуемые нами колонки
    """
    df.dropna(axis='columns', how='all', inplace=True) # удаляем пустые колонки
    df.columns = pd.Series(df.iloc[0,]).astype('str') # превращаем первую строку в заголовок таблицы
    df = cleanСolsNames(df)

    indeces2remove = []

    # находим номер колонки, в которой содержится ненужный нам номер строки или пункта, если они попали в заголовок таблицы
    i = -1
    for col in df.columns:
        i += 1
        if col.find('№') > -1:
            indeces2remove.append(i)
            break
    # или находим номер колонки, в которой содержится ненужный нам номер строки или пункта, если они не попали в заголовок и отличаются от других колонок двумя строками с NA
    header_rows = 3
    j = -1
    for index, col in df.iloc[0:header_rows].iteritems():
        j += 1
        if col.isnull().values.all():
            indeces2remove.append(j)

    # удаляем колонку с найденным индексом
    column_numbers = [x for x in range(df.shape[1])]  # list of columns' integer indices
    for index in indeces2remove:
        column_numbers.remove(index) #removing column integer index i
    df = df.iloc[:, column_numbers]
    df.columns = range(df.shape[1])
    return df

def cleanСolsNames(df):
    '''Очищает заголовок таблицы символов перевода строки, двоеточий, переводит заглавные буквы в строчные'''
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace("\n", "")
    df.columns = df.columns.str.replace(":", "")
    return df

def renameColumns(df, column_mapping):
    rename_dict = dict(zip(column_mapping['original'], column_mapping['translation']))
    df.rename(columns=rename_dict, inplace=True)
    # проверяем, что все колонки переименовались
    print('Проверяем, что все переименовано.\nЕсли ничего не печатается, то все переименовано.\nЕсли остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах')
    for column in df.columns:
        if re.search(r'[а-яА-Я]', str(column)):
            print(column)
    return df

def dropNARows(df):
    # Preview the first row that would be dropped
    rows_to_drop_first = df.iloc[0:1]
    #if not rows_to_drop_first.empty:
      #print("Rows that will be dropped (first row):")
      #print(rows_to_drop_first)
      #print(rows_to_drop_first.clause)

    # Preview rows with NaN values that would be dropped
    rows_to_drop_na = df[df.isna().any(axis=1)]
    #if not rows_to_drop_na.empty:
      #print(f"\nRows with NaN values that will be dropped ({len(rows_to_drop_na)} rows):")
      #print(rows_to_drop_na)
      #print(rows_to_drop_na.clause)

    df.drop(df.index[0:1], inplace=True)
    df.dropna(axis='rows', how='any', inplace=True)
    return df

def deleteUselessRows(df, year):
    '''Удаляем строки, в которых нет информации о конкретных статьях'''
    df['clause'] = df['clause'].astype(str)
    if 'name' in df.columns:
        df = df[~df['name'].str.contains('ВСЕГО|ИТОГО')]
    df = df.reset_index(drop=True)
    return df

def insertEmptyRows(df, year, clauses2Insert):
    '''Поскольку в 2012 и 2015 годах встречаются строки, в которых собраны несколько статей, все значения по которым равны
    нулю, т.е. не было ни осужденных, ни прекращенных дел, ни оправданных и т.д., то такие строки можно разбить по статьям
    '''
    year = str(year)
    dict2Insert = {}
    if year in clauses2Insert.keys():
        for clause in clauses2Insert[year]:
            for col in df.columns:
                dict2Insert[col] = [0]
            dict2Insert['clause'] = clause
            df2Insert = pd.DataFrame.from_dict(dict2Insert)
            df_concat = pd.concat([df, df2Insert], sort=True)
            df = df_concat.reset_index(drop=True)
    return df

def cleanClauseCol(df):
    df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в УК РФ после утверждения форм отчетности:\nКлевета \(введ. ФЗ от 28.07.2012 N 141-ФЗ\) ст. ", "")
    df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в раздел IX \"Преступления против общественной безопасности и общественного порядка\" УК РФ после утверждения форм отчетности приказом № 127\s+ст. ", "")
    df['clause'] = df['clause'].str.replace("Клевета в отношении судьи, присяжного заседателя, прокурора, следователя, лица, производящего дознание, судебного пристава \(введ. ФЗ от 28.07.2012 N 141-ФЗ\)\s+ст. ", "")

    df['clause'] = df['clause'].str.replace('Мошенничество, совершенное в сфере кредитования; при получении выплат; с использованием платежных карт; в сфере предпринимательской деят-ти; в сфере страхования; в сфере компьютерной информации \(введ. ФЗ от 29.11.2012 N 207-ФЗ\) ст\. ', '')

    df['clause'] = df['clause'].astype("str")
    df['clause'] = df['clause'].str.replace(" ", "")
    df['clause'] = df['clause'].str.replace("-", ".")
    df['clause'] = df['clause'].str.replace("ст(\.|)", "")
    df['clause'] = df['clause'].str.replace("\.ч", "ч")

    df['clause'] = df['clause'].str.replace("207\(.+\),", "")
    return df

def clauses2column(df):
    # создаем колонку part с номером статьи и части без примечаний и удаляем из нее лишние пробелы
    # df['clause'] = df['clause'].str.replace("\.\s*ч", "ч")
    # df['part'] = df.apply(lambda row: re.search(part_pattern, row['clause']).group(), axis=1)
    # на случай, если все-таки захочется вернуть воинские преступления
    df['part'] = df.apply(lambda row: re.search(part_pattern, row['clause']).group() if re.search(r'[Вв]оинск', row['clause']) is None else "Воинские преступления", axis=1)
    df['part'] = df['part'].str.replace("^ст\.\s", "")
    df['part'] = df['part'].str.replace(" ", "")
    # тут мы отрезаем номер статьи от примечаний типа "в старой редакции" - может пригодиться для целей сверки таблиц, но в целом нет
    '''
    df['clause'] = df.apply(lambda row: row['clause'].replace(re.search(part_pattern, row['clause']).group(), ""), axis=1)
    df['clause'] = df['clause'].str.replace("\sст. $", "")
    df['clause'] = df['clause'].str.replace("^\s", "")
    # колонку clause переименовываем в comments
    df.rename(index=str, columns={"clause": "comments"}, inplace=True)
    '''
    # в колонку clause складываем статьи без частей, используя ранее созданную колонку part
    df['clause'] = df.apply(lambda row: re.search(r'(\d{3}(\.\d{1}|))', row['part']).group() if re.search(r'[Вв]оинск', row['clause']) is None else "Воинские преступления", axis=1)
    #df['clause'] = df.apply(lambda row: re.search(r'(\d{3}(\.\d{1}|))', row['part']).group(), axis=1)
    df.replace("Воинскиепреступления", "Воинские преступления", inplace=True)
    return df

def addMilitaryOfences(df):
    df = pd.concat([df, pd.DataFrame([[np.nan] * df.shape[1]], columns=df.columns)], ignore_index=True)
    ind = np.where(df['name'].isna())[0][0]
    df.at[ind, 'name'] = "Воинские преступления"
    df.at[ind, 'clause'] = "Воинские преступления"
    df.at[ind, 'part'] = "Воинские преступления"
    return df

def rearrangeCols(df, firstCols):
    cols = firstCols  + [col for col in df.columns.tolist() if col not in firstCols]
    df = df[cols]
    return df

def sortTable(df):
    '''Сортирует таблицу по колонке part - со статьей и часть методом натуральной сортировки'''
    df = df.reindex(index=order_by_index(df.index, index_natsorted(df['part'])))
    df = df.reset_index(drop=True)
    return df

def tackleMergedCells(df):
    df.iloc[0:1].fillna(method='ffill', axis='columns', inplace=True)
    df.iloc[0:3].replace(to_replace=['\n', '-', '\*', '\s{3}', '\s{2}'],value=[' ', '', '', ' ', ' '], regex=True, inplace=True)
    for i in range(2, len(df.iloc[0])): # 2 - это номер колонки, с которой начинается смерживание объединенных колонок
        df.at[0, i] = str(df.iloc[0][i]).replace("nan", "") + " " + cutNumbers(df.iloc[1][i]) + " " + cutNumbers(df.iloc[2][i])

    df.columns = pd.Series(df.iloc[0,]).str.strip()
    df.reset_index(drop=True, inplace=True)
    return df

def nameSeparatedRows(df, names):
    for key in names.keys():
        ind = df.index[df['part'] == key].tolist()
        if len(ind) == 1:
            df.at[ind[0], 'name'] = names[key]
        else:
          print(f'Строка с частью {key} повторяется {len(ind)} раз, индексы {ind}')
    return df

def meltTable(df, year):
    '''Приводит таблицу в длинную форму'''
    df.insert(0, 'year', [year]*len(df))
    df = pd.melt(df, id_vars=['year', 'clause', 'part', 'name'], value_vars = list(df.columns)[4:], var_name='parameter', value_name='value')
    return df

# Проверочные функции

def checkTablesLen(main, add, parameters = None):
    if parameters is not None:
        if len(main) != len(parameters):
            print("Не ОК: длины таблиц № 10.3 и № 10.3.1 не совпадают")
        else:
            print("ОК: длины таблиц № 10.3 и № 10.3.1 совпадают")
    if len(main) != len(add):
        print("Не ОК: длины таблиц № 10.3 и № 10-а не совпадают")
    else:
        print("ОК: длины таблиц № 10.3 и № 10-а совпадают")

def checkNumbersBetweenForms(year, mainDF, addDF, parametersDF = None):
    if year > 2010:
        for i in range(len(mainDF)):
            if mainDF.iloc[i]['part'] != parametersDF.iloc[i]['part']:
                print(i, "main", mainDF.iloc[i]['part'], "parameters", parametersDF.iloc[i]['part'])

            imprisonment_sum = parametersDF.iloc[i]['primaryImprisonment1'] + \
                                                         parametersDF.iloc[i]['primaryImprisonment1_2'] + \
                                                         parametersDF.iloc[i]['primaryImprisonment2_3'] + \
                                                         parametersDF.iloc[i]['primaryImprisonment3_5'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment5_8'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment8_10'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment10_15'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment15_20']
            if mainDF.iloc[i]['primaryImprisonment'] != imprisonment_sum:
                print("Не совпадает число осужденных к лишению свободы.", "\nСтатья: ", mainDF.iloc[i]['part'], "\n10.3", "Лишение свободы всего: ", mainDF.iloc[i]['primaryImprisonment'],
                      "\n10.3.1", "Лишение свободы сумма: ", imprisonment_sum)

            if mainDF.iloc[i]['part'] != addDF.iloc[i]['part']:
                print("Не совпадают номера статей. 10.3:", mainDF.iloc[i]['part'], "10-a:", addDF.iloc[i]['part'], "Год: ", year)
            if mainDF.iloc[i]['totalConvicted'] != addDF.iloc[i]['totalConvictedMain']:
                print("Не совпадает число осужденных по основной статье.", "\nСтатья: ", mainDF.iloc[i]['part'], "\n10.3:", mainDF.iloc[i]['totalConvicted'], "\n10-a:", addDF.iloc[i]['totalConvictedMain'], "\nГод:", year)
    else:
        for i in range(len(mainDF)):
            if mainDF.iloc[i]['part'] != addDF.iloc[i]['part']:
                print("Не совпадают номера статей. 10.3:", mainDF.iloc[i]['part'], "10-a:", addDF.iloc[i]['part'], "Год: ", year)
            if mainDF.iloc[i]['totalConvicted'] != addDF.iloc[i]['totalConvictedMain']:
                print("Не совпадает число осужденных по основной статье.", "\nСтатья: ", mainDF.iloc[i]['part'], "\n10.3:", mainDF.iloc[i]['totalConvicted'], "\n10-a:", addDF.iloc[i]['totalConvictedMain'], "\nГод:", year)

def compareSums(df, columns, total_values, start):
    columns = list(columns)[start:]
    cols = [col for col in columns if not re.search(r'[а-яА-Я]', col)]
    values2remove = []
    for col in columns:
        match = re.search(r'[а-яА-Я]{2,}', col)
        if match:
            ind = columns.index(col)
            values2remove.append(total_values[ind])
    values = [val for val in total_values if val not in values2remove]
    total_values = values
    DFreordered = df[cols]
    sums = DFreordered.sum(axis='index')
#     print(sums)
    if len(total_values) != len(sums):
        print("Количество колонок не совпадает")
    for i in range(0, len(total_values)):
        if total_values[i] != sums[i]:
            print('\nСумма значений в колонке', DFreordered.columns[i], sums[i], '\n', 'Значение в строке "Всего по составам УК РФ" ', total_values[i])

In [2]:
import os
import re
import pandas as pd
import numpy as np
from natsort import natsorted, index_natsorted, order_by_index

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

import warnings
warnings.filterwarnings('ignore')

In [3]:
year = 2022

## Обработка файлов формы № 10.3

In [4]:
file_path = "ExcelFiles/Main/10.3-2022.xls"

mainDF = pd.read_excel(file_path, sheet_name=1)

# Убираем строки и колонки, не содержащие полезной информации
mainDF = dropFirstRows(mainDF, 'Статьи УК РФ')
mainDF = deleteUnusedCols(mainDF, year)

# эта строка будет нужна позже для проверки
total_values = list(mainDF[mainDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][1:])

mainDF = dropFirstRows(mainDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

# Переводим названия колонок
mainDF.iloc[1][16] = mainDF.iloc[1][15]
mainDF.iloc[1][18] = mainDF.iloc[1][17]
mainDF.iloc[1][20] = mainDF.iloc[1][19]
mainDF = tackleMergedCells(mainDF)
mainDF = cleanСolsNames(mainDF)

columns = mainDF.columns # эта строка тоже будет нужна позже для проверки

columns2eng = pd.read_csv('translations/2022/colNames2engNames_2022.csv')
columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
mainDF = renameColumns(mainDF, columns2eng)

# Убираем строки, не содержащие полезной информации
mainDF = dropNARows(mainDF)
mainDF = deleteUselessRows(mainDF, year)


# Некоторые статьи в исходных формах склеены вместе. Если по ним нет цифр (нет ни осужденных, ни оправданных и т.д.),
# то их можно безболезненно расклеить и везде вписать нули
remove_rows_with_text = ["201.2 ч. 1, 201.2 ч. 2, 201.3, 208 ч. 3",
                         "280.4 ч. 1, 280.4 ч. 2, 280.4 ч. 3, 282.4 ч. 1, 282.4 ч. 2, 283.2 ч. 1, 283.2 ч. 2, 283.2 ч. 3, 283.2 ч. 4",
                         "285.5 ч. 1, 285.5 ч. 2, 285.6, 286 ч. 4, 286 ч. 5",
                         "302 ч. 3, 302 ч. 4 ",
                         "356.1 ч. 1, 356.1 ч. 2, 356.1 ч. 3, 356.1 ч. 4"
]
for text in remove_rows_with_text:
  mainDF = mainDF[~mainDF['clause'].str.contains(text)]

clauses2Insert = {'2022': [
    '201.2 ч. 1',
    '201.2 ч. 2',
    '201.3',
    '208 ч. 3',
    '280.4 ч. 1',
    '280.4 ч. 2',
    '280.4 ч. 3',
    '282.4 ч. 1',
    '282.4 ч. 2',
    '283.2 ч. 1',
    '283.2 ч. 2',
    '283.2 ч. 3',
    '283.2 ч. 4',
    '285.5 ч. 1',
    '285.5 ч. 2',
    '285.6',
    '286 ч. 4',
    '286 ч. 5',
    '302 ч. 3',
    '302 ч. 4',
    '356.1 ч. 1',
    '356.1 ч. 2',
    '356.1 ч. 3',
    '356.1 ч. 4',

    '275.1',
    '274.2 ч. 1',

]}

mainDF = insertEmptyRows(mainDF, year, clauses2Insert)

# Некоторые статьи придется оставить склеенными :(

clause264_3 = '264.3'
part264_3 =  '264.3 ч. 1, 264.3 ч. 2'
ind = mainDF.index[mainDF['clause'] == '264.3 ч. 1, 264.3 ч. 2, 274.2 ч. 1, 275.1'][0]

mainDF = cleanClauseCol(mainDF) # чистим колонку с номерами статей от лишнего текста
mainDF = clauses2column(mainDF) # используя регулярное выражение, создаем колонки со номерами статей и частей

# после чистки подправляем те строки, которые должны остаться склеенными
mainDF.at[ind, 'clause'] = clause264_3
mainDF.at[ind, 'part'] = part264_3

firstCols = ['clause', 'part', 'totalConvicted']
mainDF = rearrangeCols(mainDF, firstCols) # меняем порядок столбцов для удобства
mainDF = sortTable(mainDF) # упорядочиваем строки

Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах


In [5]:
compareSums(mainDF, columns, total_values, 2)

In [6]:
# Для проверки, что никакие цифры не потерялись, можно посчитать суммы по столбцам и выборочно сверить с исходной таблицей (строка ВСЕГО ПО СОСТАВАМ).

mainDF.sum(axis=0)

clause                                   105105106107107108108109109109110.1110.1110.11...
part                                     105ч.1105ч.2106107ч.1107ч.2108ч.1108ч.2109ч.11...
totalConvicted                                                                      578751
acquittal                                                                             1093
addDisqualification                                                                  58577
addFine                                                                               5688
addRestrain                                                                           5821
addTitlesWithdraw                                                                      243
coerciveMeasures                                                                      7835
dismissalAbsenceOfEvent                                                                951
dismissalAmnesty                                                                        19

In [7]:
mainDF

,clause,part,totalConvicted,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,primaryArrest,primaryCommunityService,primaryCorrectionalLabour,primaryDisqualification,primaryFine,primaryForcedLabour,primaryImprisonment,primaryLifeSentence,primaryMilitaryDisciplinaryUnit,primaryOther,primaryRestrain,primaryRestrictionsInMilitaryService,primarySuspended,unfinishedOffence
0,105,105ч.1,4250,43,4,0,411,1,406,6,0,0,314,0,0,0,0,0,8,0,1,0,0,0,0,0,1,0,0,0,0,4214,0,0,0,0,0,26,731
1,105,105ч.2,1090,24,1,0,734,4,99,2,0,0,42,0,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,0,0,1045,39,0,0,0,0,3,234
2,106,106,27,1,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,6,0,6,4
3,107,107ч.1,39,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,3,0,0,0,0,0,5,0,0,0,0,0,0,2,28,1,0,2
4,107,107ч.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,360,360ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1032,361,361ч.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1033,361,361ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1034,361,361ч.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Обработка файлов формы № 10.3.1

In [8]:
parametersFile = "ExcelFiles/MainParameters/10.3.1-2022.xls"
parametersDF = pd.read_excel(parametersFile, 1, header=None)

total_values = list(parametersDF[parametersDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][3:])

parametersDF = dropFirstRows(parametersDF, 'Статьи УК РФ')
parametersDF = deleteUnusedCols(parametersDF, year)

parametersDF = dropFirstRows(parametersDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

parametersDF = tackleMergedCells(parametersDF)

parametersDF = cleanСolsNames(parametersDF)
columns2eng = pd.read_csv('translations/2022/colNames2engNamesParameters_2022.csv')
columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
parametersDF = renameColumns(parametersDF, columns2eng)

columns = parametersDF.columns

parametersDF = parametersDF[parametersDF.columns[~parametersDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
parametersDF = dropNARows(parametersDF)
parametersDF = deleteUselessRows(parametersDF, year)

for text in remove_rows_with_text:
  parametersDF = parametersDF[~parametersDF['clause'].str.contains(text)]

parametersDF = insertEmptyRows(parametersDF, year, clauses2Insert)

ind = parametersDF.index[parametersDF['clause'] == '264.3 ч. 1, 264.3 ч. 2, 274.2 ч. 1, 275.1'][0]

parametersDF = cleanClauseCol(parametersDF)
parametersDF = clauses2column(parametersDF)

parametersDF.at[ind, 'clause'] = clause264_3
parametersDF.at[ind, 'part'] = part264_3

firstCols = ['clause', 'part']
parametersDF = rearrangeCols(parametersDF, firstCols)
parametersDF = sortTable(parametersDF)

Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах
лишение свободы всего (число лиц)
общая сумма штрафов (из гр. 13 "основное наказание" ф. 10.3) всего лиц
общая сумма штрафов (из гр. 28 "дополнительное наказание" ф. 10.3) всего лиц
общая сумма судебных штрафов, назначенных в соответствии со статьей 104.4 ук рф всего лиц


In [9]:
# Незначительные расхождения это ок, т.к. скорее всего в исходных формах они тоже есть (мир суддепа несовершенен!)

compareSums(parametersDF, columns, total_values, 2)


Сумма значений в колонке dismissalCourtFine5 4832 
 Значение в строке "Всего по составам УК РФ"  4837

Сумма значений в колонке dismissalCourtFine5_25 12742 
 Значение в строке "Всего по составам УК РФ"  12744

Сумма значений в колонке dismissalCourtFineSum 337826202 
 Значение в строке "Всего по составам УК РФ"  337831202


In [10]:
parametersDF.sum(axis=0)

clause                                105105106107107108108109109109110.1110.1110.11...
part                                  105ч.1105ч.2106107ч.1107ч.2108ч.1108ч.2109ч.11...
addFine100_300                                                                      977
addFine1M                                                                           266
addFine25_100                                                                      1910
addFine300_500                                                                      452
addFine5                                                                            565
addFine500_1M                                                                       265
addFine5_25                                                                        1253
addFineSum                                                                   4240891496
dismissalCourtFine100                                                               211
dismissalCourtFine25_100        

In [11]:
parametersDF

,clause,part,addFine100_300,addFine1M,addFine25_100,addFine300_500,addFine5,addFine500_1M,addFine5_25,addFineSum,dismissalCourtFine100,dismissalCourtFine25_100,dismissalCourtFine5,dismissalCourtFine5_25,dismissalCourtFineSum,primaryFine100_300,primaryFine1M,primaryFine25_100,primaryFine300_500,primaryFine5,primaryFine500_1M,primaryFine5_25,primaryFineSum,primaryImprisonment1,primaryImprisonment10_15,primaryImprisonment15_20,primaryImprisonment1_2,primaryImprisonment2_3,primaryImprisonment3_5,primaryImprisonment5_8,primaryImprisonment8_10,primaryImprisonmentUnderLowerLimit
0,105,105ч.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,403,0,9,20,151,2129,1502,142
1,105,105ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,452,308,1,3,15,92,173,20
2,106,106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,4,1,0,0,0
3,107,107ч.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,107,107ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,360,360ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1032,361,361ч.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1033,361,361ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1034,361,361ч.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Обработка файлов формы № 10-а

In [12]:
addFile = 'ExcelFiles/Add/10a-2022.xls'

df_list = pd.read_excel(addFile, sheet_name=None, header = None)
dfs = []
for key in list(df_list.keys())[2:-3]: # -3 мы исключаем листы, где приводятся данные по пунктам статей, разберемся с ними отдельной задачей
    df = df_list[key]
    df = dropFirstRows(df, 'Виды преступлений')
    df = dropFirstRows(df, '[а-яА-Я]{3,}', 1)
    df = deleteUnusedCols(df, year)
    df = tackleMergedCells(df)
    columns2eng = pd.read_csv('translations/2022/colNames2engNamesAdd_2022.csv')
    columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
    df = renameColumns(df, columns2eng) # тут заголовок таблицы предварительно чистить не надо

    df = df[df.columns[~df.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
    df = dropNARows(df)
    dfs.append(df)
    addDF = pd.concat(dfs, ignore_index=True)

addDF.clause = addDF.clause.astype(str)

remove_rows_with_text = ["280.4 ч. 1, 280.4 ч. 2, 280.4 ч. 3, 282.4 ч. 1, 282.4 ч. 2",
                         "283.2 ч. 1, 283.2 ч. 2, 283.2 ч. 3, 283.2 ч. 4",
                         "264.3 ч. 1",
                         "264.3 ч. 2"]
for text in remove_rows_with_text:
  addDF = addDF[~addDF['clause'].str.contains(text)]

# убираем из списка статей для вставки те, что и так уже есть в форме 10а
for i in [
    '201.2 ч. 1',
    '201.2 ч. 2',
    '201.3',
    '208 ч. 3',
    '285.5 ч. 1',
    '285.5 ч. 2',
    '285.6',
    '286 ч. 4',
    '286 ч. 5',
    '302 ч. 3',
    '302 ч. 4',
    '356.1 ч. 1',
    '356.1 ч. 2',
    '356.1 ч. 3',
    '356.1 ч. 4',

    '275.1',
    '274.2 ч. 1',
    ]:
  clauses2Insert['2022'].remove(i)


clauses2Insert['2022'].append(part264_3)

addDF = insertEmptyRows(addDF, year, clauses2Insert)

ind = addDF.index[addDF['clause'] == part264_3][0]
addDF.at[ind, 'addTotalOffences'] = 1

addDF = cleanClauseCol(addDF)
addDF = clauses2column(addDF)
addDF = addMilitaryOfences(addDF)

addDF.at[ind, 'clause'] = clause264_3
addDF.at[ind, 'part'] = part264_3

addDF.clause = addDF.clause.astype(str)
addDF.name = addDF.name.astype(str)

names = {
    '116.1ч.2': 'Нанесение побоев лицом, подвергнутым административному наказанию или имеющим судимость',
    '201.2ч.1': 'Нарушение условий государственного контракта по государственному оборонному заказу либо условий договора, заключенного в целях выполнения государственного оборонного заказа',
    '201.2ч.2': 'Нарушение условий государственного контракта по государственному оборонному заказу либо условий договора, заключенного в целях выполнения государственного оборонного заказа',
    '201.3': 'Отказ или уклонение лица, подвергнутого административному наказанию, от заключения государственного контракта по государственному оборонному заказу либо договора, необходимого для выполнения государственного оборонного заказа',
    '208ч.3': 'Организация незаконного вооруженного формирования или участие в нем, а равно участие в вооруженном конфликте или военных действиях в целях, противоречащих интересам Российской Федерации',
    '274.2ч.1': 'Нарушение правил централизованного управления техническими средствами противодействия угрозам устойчивости, безопасности и целостности функционирования на территории Российской Федерации информационно-телекоммуникационной сети "Интернет" и сети связи общего пользования',
    '275.1': 'Сотрудничество на конфиденциальной основе с иностранным государством, международной либо иностранной организацией',
    '280.4ч.1': 'Публичные призывы к осуществлению деятельности, направленной против безопасности государства',
    '280.4ч.2': 'Публичные призывы к осуществлению деятельности, направленной против безопасности государства',
    '280.4ч.3': 'Публичные призывы к осуществлению деятельности, направленной против безопасности государства',
    '282.4ч.1': 'Неоднократные пропаганда либо публичное демонстрирование нацистской атрибутики или символики, либо атрибутики или символики экстремистских организаций, либо иных атрибутики или символики, пропаганда либо публичное демонстрирование которых запрещены федеральными законами',
    '282.4ч.2': 'Неоднократные пропаганда либо публичное демонстрирование нацистской атрибутики или символики, либо атрибутики или символики экстремистских организаций, либо иных атрибутики или символики, пропаганда либо публичное демонстрирование которых запрещены федеральными законами',
    '283.2ч.2': 'Нарушение требований по защите государственной тайны',
    '283.2ч.3': 'Нарушение требований по защите государственной тайны',
    '283.2ч.4': 'Нарушение требований по защите государственной тайны',
    '283.2ч.1': 'Нарушение требований по защите государственной тайны',
    '285.5ч.1': 'Нарушение должностным лицом условий государственного контракта по государственному оборонному заказу либо условий договора, заключенного в целях выполнения государственного оборонного заказа',
    '285.5ч.2': 'Нарушение должностным лицом условий государственного контракта по государственному оборонному заказу либо условий договора, заключенного в целях выполнения государственного оборонного заказа',
    '285.6': 'Отказ или уклонение должностного лица, подвергнутого административному наказанию, от заключения государственного контракта по государственному оборонному заказу либо договора, необходимого для выполнения государственного оборонного заказа',
    '286ч.4': 'Превышение должностных полномочий',
    '286ч.5': 'Превышение должностных полномочий',
    '302ч.3': 'Принуждение к даче показаний',
    '302ч.4': 'Принуждение к даче показаний',
    '356.1ч.1': 'Мародерство',
    '356.1ч.2': 'Мародерство',
    '356.1ч.3': 'Мародерство',
    '356.1ч.4': 'Мародерство',

    '264.3 ч. 1, 264.3 ч. 2': 'Управление транспортным средством лицом, лишенным права управления транспортными средствами и подвергнутым административному наказанию или имеющим судимость',
}


addDF = nameSeparatedRows(addDF, names)

ind = addDF.index[addDF['part'] == '136'][0]
addDF.at[ind, 'part'] = '136ч.2'
addDF['part'] = addDF['part'].replace('168ч.1','168')

firstCols = ['name', 'clause', 'part', 'totalConvictedMain']
addDF = rearrangeCols(addDF, firstCols)
addDF = sortTable(addDF)

Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за

In [13]:
#len(addDF[addDF.name.str.contains('Составы')])
#addDF[addDF.name == '0']

In [14]:
addDF.head()

,name,clause,part,totalConvictedMain,addAcquittalOffences,addAcquittalPersons,addDismissalOffences,addDismissalOtherOffences,addDismissalOtherPersons,addDismissalPersons,addTotalOffences,addTotalPersons,addUnfitToPleadOffences
0,Убийство,105,105ч.1,4250,7,7,0,7,7,0,71,67,12
1,Убийство при отягчающих обстоятельствах,105,105ч.2,1090,24,23,8,1,1,5,129,40,13
2,Убийство матерью новорожденного ребенка,106,106,27,0,0,0,0,0,0,0,0,0
3,"Убийство, совершенное в состоянии аффекта",107,107ч.1,39,0,0,0,1,1,0,0,0,0
4,Убийство в состоянии аффекта двух или более лиц,107,107ч.2,1,0,0,0,0,0,0,0,0,0


In [15]:
checkTablesLen(mainDF, addDF, parametersDF)
checkTablesLen(mainDF, addDF)

ОК: длины таблиц № 10.3 и № 10.3.1 совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают


In [16]:
addDF[addDF.part.str.contains('264.3')]

,name,clause,part,totalConvictedMain,addAcquittalOffences,addAcquittalPersons,addDismissalOffences,addDismissalOtherOffences,addDismissalOtherPersons,addDismissalPersons,addTotalOffences,addTotalPersons,addUnfitToPleadOffences
746,"Управление транспортным средством лицом, лишен...",264.3,"264.3 ч. 1, 264.3 ч. 2",0,0,0,0,0,0,0,1,0,0


In [17]:
mainDF[mainDF.part.str.contains('264.3')]

,clause,part,totalConvicted,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,primaryArrest,primaryCommunityService,primaryCorrectionalLabour,primaryDisqualification,primaryFine,primaryForcedLabour,primaryImprisonment,primaryLifeSentence,primaryMilitaryDisciplinaryUnit,primaryOther,primaryRestrain,primaryRestrictionsInMilitaryService,primarySuspended,unfinishedOffence
746,264.3,"264.3 ч. 1, 264.3 ч. 2",16,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,1,1,1,0,0,0,0,0,2,0


In [16]:
checkNumbersBetweenForms(year, mainDF, addDF, parametersDF)

Не совпадает число осужденных по основной статье. 
Статья:  264.3 ч. 1, 264.3 ч. 2 
10.3: 16 
10-a: 0 
Год: 2022
Не совпадает число осужденных по основной статье. 
Статья:  Воинские преступления 
10.3: 1379 
10-a: nan 
Год: 2022


In [17]:
addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
parametersDF = parametersDF.drop(['clause', 'part'], axis=1)
df = pd.concat([mainDF, parametersDF, addDF], axis=1)

In [18]:
df.head()

,clause,part,totalConvicted,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,...,primaryFine25_100,primaryFine300_500,primaryFine5,primaryFine500_1M,primaryFine5_25,primaryFineSum,primaryImprisonment1,primaryImprisonment10_15,primaryImprisonment15_20,primaryImprisonment1_2,primaryImprisonment2_3,primaryImprisonment3_5,primaryImprisonment5_8,primaryImprisonment8_10,primaryImprisonmentUnderLowerLimit,name,addAcquittalOffences,addAcquittalPersons,addDismissalOffences,addDismissalOtherOffences,addDismissalOtherPersons,addDismissalPersons,addTotalOffences,addTotalPersons,addUnfitToPleadOffences
0,105,105ч.1,4250,43,4,0,411,1,406,6,0,0,314,0,0,0,0,0,8,0,1,0,0,0,0,...,0,0,0,0,0,0,0,403,0,9,20,151,2129,1502,142,Убийство,7,7,0,7,7,0,71,67,12
1,105,105ч.2,1090,24,1,0,734,4,99,2,0,0,42,0,0,0,0,0,2,0,1,0,0,0,1,...,0,0,0,0,0,0,1,452,308,1,3,15,92,173,20,Убийство при отягчающих обстоятельствах,24,23,8,1,1,5,129,40,13
2,106,106,27,1,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,9,4,1,0,0,0,Убийство матерью новорожденного ребенка,0,0,0,0,0,0,0,0,0
3,107,107ч.1,39,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Убийство, совершенное в состоянии аффекта",0,0,0,1,1,0,0,0,0
4,107,107ч.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Убийство в состоянии аффекта двух или более лиц,0,0,0,0,0,0,0,0,0


In [19]:
dfMelted = meltTable(df, year)

In [20]:
dfMelted.head(5)

,year,clause,part,name,parameter,value
0,2022,105,105ч.1,Убийство,acquittal,43
1,2022,105,105ч.2,Убийство при отягчающих обстоятельствах,acquittal,24
2,2022,106,106,Убийство матерью новорожденного ребенка,acquittal,1
3,2022,107,107ч.1,"Убийство, совершенное в состоянии аффекта",acquittal,0
4,2022,107,107ч.2,Убийство в состоянии аффекта двух или более лиц,acquittal,0


In [21]:
dfMelted.to_pickle('2022.pkl')